In [7]:
import json
from collections import defaultdict


In [20]:
import csv
def format_entry(entry):
    # Extract components from the annotation
    annotation = entry["annotation"]
    # print(entry.keys())
    # print(entry["qa"])
    try:
        que = entry["qa"]
        print(que)
        # Build base components
        parts = [
            f"Question: {que['question']}",
            f"Pre-text: {annotation['amt_pre_text']}",
            f"Table:\n{annotation['amt_table']}",
            f"Post-text: {annotation['amt_post_text']}"
        ]
        
        # Add dialogue turns and programs
        for idx, (dialogue, program) in enumerate(zip(annotation['dialogue_break_ori'], 
                                                    annotation['turn_program'])):
            parts.append(f"Dialogue Turn {idx+1}: {dialogue}\nProgram: {program}")
        
        # Add final answer and program
        parts.append(f"The final answer is {que['answer']}")
        parts.append(f"The final program is {que['program']}")
        
        return [que['question'],que['answer'],que['program'],"\n".join(parts),"Pre-text: "+annotation['amt_pre_text']+"\n"+ "Table :"+annotation['amt_table']+"\n"+"Post-text :"+annotation['amt_post_text']]
    except:
        return []

def process_dataset(input_path, output_path):
    # Load data
    csv_data = []
    with open(input_path) as f:
        data = json.loads(f.read())
    
    # Group by filename and process
    grouped_data = defaultdict(list)
    for entry in data:
        grouped_data[entry["filename"]].append(entry)
    
    # Process each group
    processed = []
    chunk_no=0
    for filename, entries in grouped_data.items():
        # Sort entries by turn index
        # sorted_entries = sorted(entries, 
        #                       key=lambda x: x["annotation"]["turn_ind"])
        
        # # # Get the complete conversation from the entry with max turn index
        # main_entry = max(sorted_entries, 
        #                 key=lambda x: x["annotation"]["turn_ind"])
        
        # Format the entry
        print(entries)
        formatted = format_entry(entries[0])
        # Add to CSV data
        if (len(formatted)==0):
            continue
        csv_data.append({
            'Question': formatted[0],
            'Answer': formatted[1],
            'Program': formatted[2],
            "Processed":formatted[3],
            "Chunk":formatted[4],
            "Chunk_no":f"Chunk_{chunk_no}"
        })
        chunk_no=chunk_no+1

    
    # Save to file
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['Question', 'Answer', 'Program',"Processed","Chunk","Chunk_no"])
            writer.writeheader()
            writer.writerows(csv_data)


In [ ]:
process_dataset(r"C:\Users\hp\OneDrive\Desktop\Tomoro AI\Correct_Format\train_turn.json", 'test1.csv')

In [1]:
import pandas as pd
df=pd.read_csv("test1.csv")

In [2]:
df.head()

,Question,Answer,Program,Processed,Chunk,Chunk_no
0,what was the percentage change in the net cash...,14.1%,"subtract(206588, 181001), divide(#0, 181001)",Question: what was the percentage change in th...,Pre-text: 26 | 2009 annual report in fiscal 20...,Chunk_0
1,what was the percent of the growth in the reve...,1.3%,"subtract(9362.2, 9244.9), divide(#0, 9244.9)",Question: what was the percent of the growth i...,Pre-text: substantially all of the goodwill an...,Chunk_1
2,what was the percentage change in net sales fr...,-32%,"subtract(5363, 7983), divide(#0, 7983)",Question: what was the percentage change in ne...,Pre-text: in a new business model such as the ...,Chunk_2
3,what was the difference in percentage cumulati...,-26.16%,"subtract(75.95, const_100), divide(#0, const_1...",Question: what was the difference in percentag...,Pre-text: ( 1 ) includes shares repurchased th...,Chunk_3
4,what portion of the total shares subject to ou...,70.1%,"add(2530454, 5923147), divide(5923147, #0)",Question: what portion of the total shares sub...,Pre-text: tax returns for 2001 and beyond are ...,Chunk_4


In [3]:
df.tail()

,Question,Answer,Program,Processed,Chunk,Chunk_no
629,what is the percent change in weighted average...,-9%,"subtract(215.0, 235.5), divide(#0, 235.5)",Question: what is the percent change in weight...,Pre-text: 14 . capital stock and earnings per ...,Chunk_629
630,how much percentage has long-term debt gone do...,99.6%,"subtract(244.5, 0.9), divide(#0, 244.5)",Question: how much percentage has long-term de...,Pre-text: notes to consolidated financial stat...,Chunk_630
631,what was the increase in the interest expenses...,8.21%,"divide(130.5, 120.6), subtract(#0, const_1)",Question: what was the increase in the interes...,Pre-text: goodwill and intangible asset impair...,Chunk_631
632,what is the total value of securities approved...,61.65,"multiply(1765510, 34.92), divide(#0, const_100...",Question: what is the total value of securitie...,Pre-text: item 11 2014executive compensation w...,Chunk_632
633,what was the percentage increase in the total ...,2.6%,"subtract(7.8, 7.6), divide(#0, 7.6)",Question: what was the percentage increase in ...,Pre-text: on-balance sheet securitizations the...,Chunk_633
